In [40]:
from fastai.tabular import *  
import pandas as pd
from pathlib import Path
from fastai import *

path = Path("/home/omar/Desktop/heartfile")

In [93]:
dfdata = pd.read_csv('./train_values.csv')
dftest = pd.read_csv('./test_values.csv')
dfavecnom=dftest
dfrealtest=dftest.drop(columns=['patient_id'])
dflabels = pd.read_csv('./train_labels.csv')
df = pd.merge(dfdata, dflabels, on='patient_id')
df=df.drop(columns=['patient_id'])

In [42]:
df.head()

slope_of_peak_exercise_st_segment    thal  resting_blood_pressure  \
0                                  1  normal                     128   
1                                  1  normal                     128   
2                                  1  normal                     128   
3                                  1  normal                     128   
4                                  1  normal                     128   

   chest_pain_type  num_major_vessels  fasting_blood_sugar_gt_120_mg_per_dl  \
0                2                  0                                     0   
1                2                  0                                     0   
2                2                  0                                     0   
3                2                  0                                     0   
4                2                  0                                     0   

   resting_ekg_results  serum_cholesterol_mg_per_dl  oldpeak_eq_st_depression  \
0                    2                          308                       0.0   
1                    2                          308                       0.0   
2                    2                          308                       0.0   
3                    2                          308                       0.0   
4                    2                          308                       0.0   

   sex  age  max_heart_rate_achieved  exercise_induced_angina  \
0    1   45                    170.0                      0.0   
1    1   45                    170.0                      0.0   
2    1   45                    170.0                      0.0   
3    1   45                      NaN                      NaN   
4    1   45                      NaN                      NaN   

   heart_disease_present  
0                      0  
1                      0  
2                      0  
3                      0  
4                      0

In [43]:
procs = [FillMissing, Categorify, Normalize]

In [44]:
valid_idx = range(len(df)-165, len(df))

In [45]:
dep_var = 'heart_disease_present'
cat_names = ['thal']

In [46]:
data = TabularDataBunch.from_df(path, df, dep_var, valid_idx=valid_idx, procs=procs, cat_names=cat_names)
print(data.train_ds.cont_names)

['slope_of_peak_exercise_st_segment', 'num_major_vessels', 'resting_ekg_results', 'age', 'serum_cholesterol_mg_per_dl', 'exercise_induced_angina', 'oldpeak_eq_st_depression', 'resting_blood_pressure', 'chest_pain_type', 'fasting_blood_sugar_gt_120_mg_per_dl', 'sex', 'max_heart_rate_achieved']


In [153]:
learn = tabular_learner(data, layers=[200,100], emb_szs={'thal':10}, metrics=accuracy)
learn.fit_one_cycle(1, 1e-2)
#learn.fit(1, 1e-2)

In [155]:
result=[]
for i in range(90):
    result.append({'patient_id':dfavecnom.iloc[i,0],'heart_disease_present':float(learn.predict(dfrealtest.iloc[i])[2][1])}) 
    

In [165]:
result = pd.DataFrame(result)
result=result.iloc[:, ::-1]
result

patient_id  heart_disease_present
0      olalu7               0.313708
1      z9n6mx               0.094412
2      5k4413               0.829184
3      mrg7q5               0.218593
4      uki4do               0.778049
5      kev1sk               0.107967
6      9n6let               0.291309
7      jxmtyg               0.832985
8      51s2ff               0.203077
9      wi9mcs               0.180039
10     741h4l               0.460007
11     1ef64a               0.553841
12     wa2ix6               0.239674
13     8167zl               0.817129
14     n6nldr               0.155223
15     ph85fp               0.119634
16     jfan5p               0.068537
17     7c4iz1               0.113878
18     ukigml               0.821859
19     flwvnq               0.108369
20     5i4fw2               0.864688
21     du1pqf               0.212720
22     vs68qz               0.144685
23     pfyez0               0.081575
24     azvkw2               0.359269
25     cird1i               0.752276
26     3bg32t               0.431076
27     xzd050               0.247003
28     eyi8et               0.441120
29     ce4x2h               0.100470
..        ...                    ...
60     tgpy9u               0.244521
61     pf5wp6               0.816237
62     cj8vj2               0.186272
63     9w6d9j               0.821551
64     3l89wd               0.143665
65     83a6x1               0.597720
66     oua0gr               0.482249
67     j0hl96               0.369780
68     dlkzyg               0.560989
69     r0w4a8               0.549533
70     46dlca               0.205842
71     9fkefu               0.863878
72     6uk6kl               0.764576
73     c7olxr               0.900945
74     iiyx0q               0.797638
75     25vetx               0.858463
76     073vc5               0.723615
77     18abn0               0.616355
78     v5fsfs               0.242074
79     2ekoo7               0.509786
80     5bbknr               0.432671
81     hr6pjx               0.075767
82     r4hsar               0.424410
83     4cezdf               0.704769
84     palhcc               0.382257
85     bwoyg6               0.209671
86     j8i7ve               0.623168
87     t2zn1n               0.169802
88     oxf8kj               0.232815
89     aeiv0y               0.238352

[90 rows x 2 columns]

In [166]:
result.round(3)
result.to_csv("results3.csv",index=False)